In [2]:
import igl
#import meshplot as mp
import vedo as vd
import pandas as pd 
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

In [3]:
def unormalize_dir(h_pts, dual, inner_vertices, tv, e_i, rad):
    """Input 
        h_pts: sphere centers
        e_i: edge directions normalized
        rad: sphere radii
        Ouput:
        le: unormalized edge directions

    """
    # factors
    l = np.ones(len(e_i))
    for i in range(len(inner_vertices)):
        # Get dual faces index
        idx = inner_vertices[i]
   
        # Get dual face
        f = dual[idx]
        
        # Get sphere centers
        p = h_pts[f]
        
        # Get edge direction
        e = e_i[idx]

        # Get radius
        r = rad[idx]

        # Direction from v_i to center
        dc = p - tv[idx]
        dc /= np.linalg.norm(dc, axis=1)[:, None] # normalize
        
        # cos angle e_i with the direction to the center
        theta = np.sum(e*dc, axis=1)

        # Get the lambdaz
        l[idx] = np.mean(2*r*theta)
        
    # unormalize
    le = e_i*l[:, None]
    return le

def init_test_data(data):
    # # Define paths
    dir_path = os.getcwd()
    data_path = dir_path+"/approximation/data/" # data path

    # Data of interest
    k = data

    # Load M mesh (centers of sphere mesh)
    mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

    # Load test mesh
    tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

    # Create dual mesh
    tmesh = Mesh()
    tmesh.make_mesh(tv,tf)

    # Get inner vertices
    inner_vertices = tmesh.inner_vertices()

    # Get vertex normals for test mesh
    e_i = igl.per_vertex_normals(tv, tf)

    # Fix normal directions
    signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
    e_i = e_i * signs[:, None]

    # Compute circumcenters and axis vectors for each triangle
    p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

    ct, _, nt = circle_3pts(p1, p2, p3)

    # Dual topology 
    dual_tf = tmesh.vertex_ring_faces_list()

    # Create hexagonal mesh                            
    h_pts = np.empty((len(tf), 3), dtype=np.float64)
    center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

    # Intersect circumcircle axis with center mesh
    for i in range(len(tf)):
        # Get points on circumcircle axis
        p0  = ct[i] - 10*nt[i]
        p1  = ct[i] + 10*nt[i]
        
        # Get intersection points
        h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

    # Get radius of spheres
    r = np.linalg.norm(h_pts - tv[tf[:,0]], axis=1)

    return tv, tf, inner_vertices, e_i, h_pts, dual_tf, r 


def compute_disc(tv, tf, e_i):

    # # Compute the edge vectors per each face
    vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]

    # # Compute the edge vectors per each face
    vij = vj - vi
    vik = vk - vi

    # Set up X 
    eij = e_i[tf[:,1]] - e_i[tf[:,0]]
    eik = e_i[tf[:,2]] - e_i[tf[:,0]]

    ec = np.sum( e_i[tf], axis = 1) / 3

    # A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
    # A = gamma11 +  gamma12
    eikXec = np.cross(eik, ec)
    vikXec = np.cross(vik, ec)

    det1 = np.sum(vij*eikXec, axis=1)
    det2 = np.sum(eij*vikXec, axis=1)

    # b = [eij, eik, ec]  c = [vij, vik, ec]

    gamma0 = np.sum(eij*eikXec, axis=1)
    gamma2 = np.sum(vij*vikXec, axis=1)

    A = det1 + det2 

    return A, A**2 - 4*gamma0*gamma2

def unormalize_dir(h_pts, dual, inner_vertices, tv, e_i, rad):
    """Input 
        h_pts: sphere centers
        e_i: edge directions normalized
        rad: sphere radii
        Ouput:
        le: unormalized edge directions

    """
    le = np.ones_like(e_i)
    for i in range(len(inner_vertices)):
        # Get dual faces index
        idx = inner_vertices[i]

        # Get dual face
        f = dual[idx]

        # Get sphere centers
        p = h_pts[f]

        # Get edge direction
        e = e_i[idx]

        # Get radius
        r = rad[idx]

        # angle e_i with the direction to the center
        theta = np.arccos(np.sum(e*(p - tv[idx]), axis=1))

        print(theta)

        # Get the lambda
        le[idx] = 2*r*np.cos(theta)

    return le

# LC + Hyp energy Minimization

In [4]:
# Init data 
tv, tf, inner_vertices, e_i, h_pts, dual_tf, r = init_test_data(4)

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Init directions
X[:3*len(tv)] = e_i.flatten()

# Init LineCong
linecong = LineCong()
linecong.initialize_constraint(X, len(tv), h_pts, dual_tf, inner_vertices, 1)

# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i, 1, 1)

# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# Time tracker
time_opt = []

for i in range(14):

    # Take time
    start = time.time()

    # Compute J, r per energy
    hyp.compute(X, tf)
    linecong.compute(X, inner_vertices, dual_tf)

    # Add constraints to optimizer
    opt.add_constraint(hyp)
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")

    # Take time
    end = time.time()

    # Update X
    X = opt.X

Mesh Data Structure: |V| = 1659, |F| = 3171, |E| = 4829
 E 1: 3472.4230536700006
 E 2: 1079.807646955132
 E 3: 336.93556708371284
 E 4: 105.80503265158106
 E 5: 33.62082382868519
 E 6: 11.01720978567038
 E 7: 3.9245360939743623
 E 8: 1.695374359856471
 E 9: 0.993913700076041
 E 10: 0.7729931077654261
 E 11: 0.7033828919503261
 E 12: 0.6814490235208466
 E 13: 0.6745416615164265
 E 14: 0.6723691385804018


In [5]:
e_i = opt.X[:3*len(tv)].reshape(len(tv), 3)
e_i /= np.linalg.norm(e_i, axis=1)[:, None]

#le = unormalize_dir(h_pts, dual_tf, inner_vertices, tv, e_i, r)

## Save data

In [9]:
def export_torsal_direction(data, tv, tf, e_i):

    # Data 
    k = data

    # Create file for positions
    pos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
    # Create file for torsal directions
    tdir = open(os.path.join( out_path, "torsal_dir_"+str(k)+".dat"), "w")
    # Create file to store the ids of the inner faces
    f_if = open(os.path.join( out_path, "inner_faces_"+str(k)+".dat"), "w")

    # Compute torsal directions
    barycenters, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)
    
    # Get inner faces
    t_mesh = Mesh()
    t_mesh.make_mesh(tv, tf)
    inner_faces = t_mesh.inner_faces()

    for ii in range(len(inner_faces)):
        i = inner_faces[ii]
        pos.write(str(barycenters[i][0]) + " " + str(barycenters[i][1]) + " " + str(barycenters[i][2]) + "\n")
        tdir.write(str(t1[i][0]) + " " + str(t1[i][1]) + " " + str(t1[i][2]) + " " + str(t2[i][0]) + " " + str(t2[i][1]) + " " + str(t2[i][2]) + "\n")
        f_if.write(str(i) + "\n")


    pos.close()
    print("Positions file created")
    tdir.close()
    print("Torsal dir files created")
    f_if.close()
    print("Inner faces file created")

def export_toral_data_testing(data, tv, tf, e_i, list_bad_faces):

    # Data
    k = data 

    # Compute torsal directions
    _, t1, t2, _ = torsal_dir_vec(tv, tf, e_i)

    # Create data frame
    df = pd.DataFrame()

    # Create a table with e_i[tf[list_bad_faces]], tv[tf[list_bad_Faces]], t1[list_bad_faces], t2[list_bad_faces]
    df["e_i"] = e_i[tf[list_bad_faces]].tolist()
    df["tv"] = tv[tf[list_bad_faces]].tolist()
    df["t1"] = t1[list_bad_faces].tolist()
    df["t2"] = t2[list_bad_faces].tolist()

    # Export df to csv
    df.to_csv(os.path.join( out_path, "bf_data_"+str(k)+".csv"), index=False)




In [10]:
export_torsal_direction(4, tv, tf, e_i)

Mesh Data Structure: |V| = 1659, |F| = 3171, |E| = 4829
Positions file created
Torsal dir files created
Inner faces file created


In [8]:
export_toral_data_testing(4, tv, tf, e_i, np.array([847,833,806,776,770,767,733,721,688,571]))